

```

1.   Praise T Ganyiwa R204436Q

2.   Fastion Mateveva R205684K

3.   Shamiso Makainganwa R204437W


```



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path =os.listdir ('/content/drive/MyDrive/dataset/train')

label_types = os.listdir ('/content/drive/MyDrive/dataset/train')
print (label_types)  

['dance', 'amapiano', 'petit']


In [ ]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/dataset/train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/dataset/train' + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

        tag                                         video_name
0     dance  /content/drive/MyDrive/dataset/train/dance/vid...
1     dance  /content/drive/MyDrive/dataset/train/dance/vid...
2     dance  /content/drive/MyDrive/dataset/train/dance/vid...
3  amapiano  /content/drive/MyDrive/dataset/train/amapiano/...
4  amapiano  /content/drive/MyDrive/dataset/train/amapiano/...
        tag                                         video_name
4  amapiano  /content/drive/MyDrive/dataset/train/amapiano/...
5  amapiano  /content/drive/MyDrive/dataset/train/amapiano/...
6     petit  /content/drive/MyDrive/dataset/train/petit/vid...
7     petit  /content/drive/MyDrive/dataset/train/petit/vid...
8     petit  /content/drive/MyDrive/dataset/train/petit/vid...


In [ ]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [ ]:
dataset_path = os.listdir('/content/drive/MyDrive/dataset/test')
print(dataset_path)

room_types = os.listdir('/content/drive/MyDrive/dataset/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/dataset/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/dataset/test' + '/' +item) + '/' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['amapiano', 'petit', 'dance']
Types of activities found:  3
        tag                                         video_name
0  amapiano  /content/drive/MyDrive/dataset/test/amapiano/v...
1     petit  /content/drive/MyDrive/dataset/test/petit/vid7...
2     dance  /content/drive/MyDrive/dataset/test/dance/vid4...
        tag                                         video_name
0  amapiano  /content/drive/MyDrive/dataset/test/amapiano/v...
1     petit  /content/drive/MyDrive/dataset/test/petit/vid7...
2     dance  /content/drive/MyDrive/dataset/test/dance/vid4...


In [ ]:
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(9)

Total videos for training: 9
Total videos for testing: 3


,Unnamed: 0,video_name,tag
4,4,/content/drive/MyDrive/dataset/train/amapiano/...,amapiano
7,7,/content/drive/MyDrive/dataset/train/petit/vid...,petit
3,3,/content/drive/MyDrive/dataset/train/amapiano/...,amapiano
0,0,/content/drive/MyDrive/dataset/train/dance/vid...,dance
6,6,/content/drive/MyDrive/dataset/train/petit/vid...,petit
5,5,/content/drive/MyDrive/dataset/train/amapiano/...,amapiano
1,1,/content/drive/MyDrive/dataset/train/dance/vid...,dance
8,8,/content/drive/MyDrive/dataset/train/petit/vid...,petit
2,2,/content/drive/MyDrive/dataset/train/dance/vid...,dance


In [ ]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['amapiano', 'dance', 'petit']


array([[1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [2],
       [2],
       [2]])

In [ ]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

1/1 [==============================] - 0s 158ms/step
Frame features in train set: (9, 20, 2048)
Frame masks in train set: (9, 20)
train_labels in train set: (9, 1)
test_labels in train set: (3, 1)


In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 10
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 1.0434 - accuracy: 0.1667
Epoch 1: val_loss improved from inf to 1.72131, saving model to ./tmp/video_classifier
1/1 [==============================] - 11s 11s/step - loss: 1.0434 - accuracy: 0.1667 - val_loss: 1.7213 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 1.0127 - accuracy: 0.6667
Epoch 2: val_loss did not improve from 1.72131
1/1 [==============================] - 0s 103ms/step - loss: 1.0127 - accuracy: 0.6667 - val_loss: 1.8808 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.9670 - accuracy: 0.5000
Epoch 3: val_loss did not improve from 1.72131
1/1 [==============================] - 0s 64ms/step - loss: 0.9670 - accuracy: 0.5000 - val_loss: 1.8837 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.6450 - accuracy: 0.8333
Epoch 4: val_loss did not improve from 1.72131
1/1 [==

In [22]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)


Test video path: /content/drive/MyDrive/dataset/test/amapiano/vid2.mp4
1/1 [==============================] - 0s 25ms/step
  dance: 54.76%
  amapiano: 27.27%
  petit: 17.97%


In [23]:
sequence_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 20, 2048)]   0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 gru (GRU)                      (None, 20, 16)       99168       ['input_4[0][0]',                
                                                                  'input_5[0][0]']                
                                                                                                  
 gru_1 (GRU)                    (None, 8)            624         ['gru[0][0]']                

In [24]:
sequence_model.save('model.h5') 

In [25]:
new_model = tf.keras.models.load_model('model.h5')

# Show the model architecture
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 20, 2048)]   0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 gru (GRU)                      (None, 20, 16)       99168       ['input_4[0][0]',                
                                                                  'input_5[0][0]']                
                                                                                                  
 gru_1 (GRU)                    (None, 8)            624         ['gru[0][0]']                